In [13]:
import scanpy as sc
import pandas as pd

In [14]:
adata = sc.read_h5ad("/wynton/home/yang/apennacchio/scGPT/data/merged.h5ad")
adata

AnnData object with n_obs × n_vars = 1376453 × 17818
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'background_fraction', 'cell_probability', 'cell_size', 'droplet_efficiency', 'pANN_0.25_0.1_6888', 'old_id', 'Sample_ID', 'CoLabs_ID', 'Pool_ID', 'Probe', 'Region', 'Purification', 'ProbeInput', 'EncapsulationInput', 'Goop', 'SampleEncapsulated', 'ExpectedRecovery', 'ActualRecovery', 'PercentRecovery', 'Category', 'Group', 'APOE', 'Age', 'Sex', 'Race', 'Source', 'Donor_ID', 'pool', 'pANN_0.25_0.001_6163', 'pANN_0.25_0.01_4537', 'pANN_0.25_0.005_13786', 'pANN_0.25_0.005_8409', 'pANN_0.25_0.01_10704', 'pANN_0.25_0.03_8719', 'subset', 'pANN_0.25_0.01_1465', 'pANN_0.25_0.05_7149', 'pANN_0.25_0.01_11086', 'pANN_0.25_0.05_9536', 'pANN_0.25_0.001_27080', 'pANN_0.25_0.04_11267', 'pANN_0.25_0.005_10916', 'pANN_0.25_0.05_9395', 'pANN_0.25_0.02_7520', 'pANN_0.25_0.07_11223', 'pANN_0.25_0.01_9459', 'S.Score', 'G2M.Score', 'Phase', 'RNA_snn_res.0.4', 'seurat_clusters', 'louvain_r

In [15]:
# load in CT labels for clusters
labels = pd.read_csv("/wynton/home/yang/apennacchio/scGPT/data/XNEU2-B1_cb_louvain_res1.5_ann.csv")
labels

,cluster,res1.5_ann
0,0,Neuron
1,1,Oligodendrocyte
2,2,Oligodendrocyte
3,3,Oligodendrocyte
4,4,Oligodendrocyte
...,...,...
69,69,Oligodendrocyte
70,70,Fibroblast
71,71,OPCs
72,72,Neuron


In [16]:
#attach CT labels and rename columns
labels["cluster"] = labels["cluster"].astype("int")
adata.obs["louvain_res1.5"] = adata.obs["louvain_res1.5"].astype("int")
adata.obs = pd.merge(adata.obs, labels, left_on='louvain_res1.5', right_on='cluster', how='left')
adata.obs["louvain_res1.5"] = adata.obs["louvain_res1.5"].astype("category")
adata.obs.rename(columns={"res1.5_ann": "celltype"}, inplace=True)
#adata.obs["str_batch"] = adata.obs["Source"].astype(str) # create copy of batch column
adata.obs["str_batch"] = adata.obs["pool"].astype(str) # create copy of batch column



In [47]:
# note that this is only needed for the CT annotation workflow
'''
# randomly select 80% of rows to be train data (0), other 20% rows are test (1)
import numpy as np

train_test_assignment = np.random.choice([0, 1], size=adata.obs.shape[0], p=[0.8, 0.2])

# Assign the batch_assignment array to the 'batch' column in the DataFrame
adata.obs['train_test_split'] = train_test_assignment
adata.obs
'''

"\n# randomly select 80% of rows to be train data (0), other 20% rows are test (1)\nimport numpy as np\n\ntrain_test_assignment = np.random.choice([0, 1], size=adata.obs.shape[0], p=[0.8, 0.2])\n\n# Assign the batch_assignment array to the 'batch' column in the DataFrame\nadata.obs['train_test_split'] = train_test_assignment\nadata.obs\n"

In [17]:
adata.write_h5ad("../data/final.h5ad")

In [18]:
#split data in half for trial run 
import numpy as np
bdata = sc.read_h5ad("../data/final.h5ad")
bdata.shape[0]
random_indices = np.random.choice(bdata.shape[0], size=bdata.shape[0] // 2, replace=False)
bdata_half = bdata[random_indices]
bdata_half.shape
bdata_half.write_h5ad("../data/half_final.h5ad")

/wynton/home/yang/apennacchio/.local/lib/python3.12/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
